In [1]:
# отключаем предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# импортируем библиотеку pandas
import pandas as pd

# cчитываем данные и выводим первые 5 наблюдений
gl = pd.read_csv('Notebooks/Data/game_logs.csv')
gl.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


Мы можем воспользоваться методом **.info()**, чтобы получить основные сведения о нашем
датафрейме, включая его размер, информацию о типах данных и использовании памяти.

По умолчанию pandas вычисляет лишь приблизительный объем памяти, используемый
датафреймом, в целях экономии времени. Поскольку нас интересует
точность, мы установим параметру **memory_usage значение 'deep'**, чтобы получить
точную информацию.

In [2]:
# выводим точную информацию об использовании памяти
gl.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 859.4 MB


In [3]:
# посмотрим, сколько памяти в среднем используют столбцы определенного типа
for dtype in ['float', 'int', 'object']:
    selected_dtype = gl.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print(f'Использование памяти в среднем для столбцов: {dtype}: {mean_usage_mb} MB')

Использование памяти в среднем для столбцов: float: 1.294733194204477 MB
Использование памяти в среднем для столбцов: int: 1.1242000034877233 MB
Использование памяти в среднем для столбцов: object: 9.500870656363572 MB


#### Подтипы
Многие типы в библиотеке pandas имеют несколько подтипов, которые могут
использовать меньшее количество байтов для представления каждого значения.
Например, тип float имеет подтипы float16, float32 и float64. Число, следующее
после названия типа, указывает количество битов, которое используется данным
типом для представления значений. Например, подтипы, которые мы только что
указали, используют соответственно 2, 4, 8 и 16 байтов.

Мы можем воспользоваться классом **numpy.iinfo**, чтобы посмотреть минимальное и максимальное значения для каждого целочисленного подтипа.

In [4]:
# импортируем библиотеку numpy
import numpy as np

# посмотрим минимальное и максимальное значения для каждого целочисленного типа
int_types = ['uint8', 'int8', 'int16']
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------



#### Оптимизация числовых столбцов с помощью понижающего преобразования
Мы можем использовать функцию <mark>pd.to_numeric()</mark>, чтобы выполнить понижающее
преобразование наших числовых типов. Мы воспользуемся DataFrame.select_
dtypes для выбора только целочисленных столбцов, тогда мы будем оптимизировать
типы и сравнивать использование памяти.

In [5]:
# мы будем довольно часто подсчитывать использование памяти,
# поэтому напишем функцию, которая сэкономит нам немного времени
def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:  # предположим, что если это не датафрейм, то серия
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return f'{usage_mb} mb'

In [6]:
# выполняем понижающее преобразование для столбцов типа int
gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric, downcast='unsigned')

print(mem_usage(gl_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([gl_int.dtypes,
                          converted_int.dtypes], axis='columns')
compare_ints.columns = ['before', 'after']
compare_ints.apply(pd.Series.value_counts)

7.8694000244140625 mb
1.475611686706543 mb


,before,after
uint8,NaN,5.0
uint32,NaN,1.0
int64,6.0,NaN


In [7]:
# выполняем понижающее преобразование для столбцов типа float
gl_float = gl.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric, downcast='float')

print(mem_usage(gl_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([gl_float.dtypes, converted_float.dtypes], axis='columns')
compare_floats.columns = ['before', 'after']
compare_floats.apply(pd.Series.value_counts)

100.98918914794922 mb
50.49465560913086 mb


,before,after
float32,NaN,77.0
float64,77.0,NaN


Давайте создадим копию нашего исходного датафрейма, заменив исходные числовые столбцы оптимизированными числовыми столбцами, и посмотрим, каким будет общее использование памяти.

In [8]:
# создаем копию исходного датафрейма
optimized_gl = gl.copy()

# заменяем исходные числовые столбцы оптимизированными
optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float

# смотрим использование памяти
print(mem_usage(gl))
print(mem_usage(optimized_gl))

859.4271268844604 mb
802.5388050079346 mb


Хотя мы значительно сократили использование памяти нашими числовыми
столбцами, в целом же мы уменьшили потребление памяти нашим датафреймом
только на **7 %**. В значительной мере снижение использования памяти будет
зависеть от оптимизации столбцов типа object.

#### Сравнение способов хранения числовых и строковых значений

Тип **object** представляет значения, использующие питоновские объекты-строки, отчасти
это обусловлено отсутствием поддержки пропущенных строковых значений в NumPy.
Поскольку Python является высокоуровневым, интерпретируемым
языком, он не предполагает точной настройки способа хранения значений в памяти.

Это ограничение приводит к тому, что строки хранятся фрагментированно, это потребляет
больше памяти и замедляет доступ. **Каждый элемент в столбце типа object является,
по сути, указателем, который содержит «адрес» фактического значения в памяти.**

In [9]:
# смотрим размеры отдельно хранящихся строк
from sys import getsizeof

s1 = 'working out'
s2 = 'memory usage for'
s3 = 'strings in python is fun!'
s4 = 'strings in python is fun!'

for s in [s1, s2, s3, s4]:
    print(getsizeof(s))

60
65
74
74


In [10]:
# смотрим размеры строк, являющихся элементами серии
obj_series = pd.Series(['working out',
                        'memory usage for',
                        'strings in python is fun!',
                        'strings in python is fun!'
                       ])
obj_series.apply(getsizeof)

0    60
1    65
2    74
3    74
dtype: int64

**Видно, что размер строк, хранящихся в объекте Series, идентичен размеру этих
же строк, хранящихся отдельно в Python.**

#### Оптимизация типов object с помощью типа category
Тип **category** под капотом для представления значений в столбце вместо исходных
использует целочисленные значения. Для этого создается отдельный словарь,
в котором исходным значениям сопоставлены целочисленные значения.
Это сопоставление будет полезно там, где столбец содержит ограниченный набор
значений. Когда мы конвертируем столбец в тип category, pandas использует самый
эффективный с точки зрения занимаемого места подтип int, который может
представить все уникальные значения в столбце.**(стр. 350)**

In [11]:
# смотрим количество уникальных значений по каждому столбцу типа object
gl_obj = gl.select_dtypes(include=['object']).copy()
gl_obj.describe()

,day_of_week,v_name,v_league,h_name,h_league,day_night,completion,forefeit,protest,park_id,...,h_player_6_id,h_player_6_name,h_player_7_id,h_player_7_name,h_player_8_id,h_player_8_name,h_player_9_id,h_player_9_name,additional_info,acquisition_info
count,171907,171907,171907,171907,171907,140150,116,145,180,171907,...,140838,140838,140838,140838,140838,140838,140838,140838,1456,140841
unique,7,148,7,148,7,2,116,3,5,245,...,4774,4720,5253,5197,4760,4710,5193,5142,332,1
top,Sat,CHN,NL,CHN,NL,D,"19200904,,0,6,36",H,V,STL07,...,grimc101,Charlie Grimm,grimc101,Charlie Grimm,lopea102,Al Lopez,spahw101,Warren Spahn,HTBF,Y
freq,28891,8870,88866,9024,88867,82724,1,69,90,7022,...,427,427,491,491,676,676,339,339,1112,140841


Прежде чем погрузиться в детали, мы просто выберем один из столбцов типа
object и посмотрим, что происходит под капотом, когда мы преобразуем его 
в категориальный тип. Мы воспользуемся вторым столбцом нашего набора данных
под названием day_of_week.

Взглянув на таблицу выше, мы видим, что он содержит лишь **семь уникальных
значений**. Мы преобразуем его в тип category, используя метод <mark>.astype()</mark>.

In [12]:
# запишем столбец day_of_week как dow
dow = gl_obj['day_of_week']
print(dow.head())

# присваиваем столбцу dow тип category
dow_cat = dow.astype('category')
print(dow_cat.head())

# просмотр категорий
dow_cat.cat.categories

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: object
0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: category
Categories (7, object): ['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed']


Index(['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed'], dtype='object')

In [13]:
# выводим целочисленные значения, соответствующие исходным значениям
dow_cat.cat.codes

0         4
1         0
2         2
3         1
4         5
         ..
171902    3
171903    3
171904    3
171905    3
171906    3
Length: 171907, dtype: int8

Видно, что каждому уникальному значению присваивается целое число, а базовым типом
данных для столбца будет тип **int8**. Этот столбец не имеет пропусков, но если они будут,
подтип category обрабатывает пропущенные значения, присвоив им значение –1.

In [14]:
# смотрим, сколько памяти использует столбец dow до и после преобразования в тип category
print(mem_usage(dow))
print(mem_usage(dow_cat))

9.836719512939453 mb
0.16475200653076172 mb


Преобразование всех столбцов в тип category – привлекательная идея, однако
важно помнить про ограничения, связанные с этой операцией. 
**Самое большое
ограничение** – невозможность вычислений с помощью численных методов. Мы
не можем выполнять арифметические операции со столбцами типа category или
использовать такие методы, как .min() и .max(), не присвоив столбцам настоящий
числовой тип.

Мы должны придерживаться типа category преимущественно при работе с такими
столбцами object, в которых менее 50 % значений являются уникальными.

Если все значения в столбце являются уникальными, тип category в конечном итоге
будет использовать больший объем памяти. Это обусловлено тем, что в столбце, помимо
целочисленных кодов, представляющих категории, хранятся все исходные строковые значения.

In [15]:
converted_obj = pd.DataFrame()

# пишем цикл, которой перебирает каждый столбец object, проверяет его на соответствие
# заданному порогу (количество уникальных значений должно быть меньше 50 %
# от общего количества значений) и, если столбец удовлетворяет порогу, преобразовывает его
# в тип category
for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:, col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:, col] = gl_obj[col]

In [16]:
# снова применяем функцию mem_usage, смотрим, сколько памяти занимают все столбцы типа object
# до и после преобразования в тип category
print(mem_usage(gl_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([gl_obj.dtypes, converted_obj.dtypes], axis='columns')
compare_obj.columns = ['before', 'after']
compare_obj.describe()

750.5687818527222 mb
49.889652252197266 mb


,before,after
count,78,78
unique,1,77
top,object,category
freq,78,2


In [17]:
# смотрим, сколько памяти использует датафрейм после оптимизации типов
optimized_gl[converted_obj.columns] = converted_obj
mem_usage(optimized_gl)

'101.85967540740967 mb'

Существует тип **datetime**, который можно использовать для представления
значений, записанных в первом столбце нашего набора данных.

In [18]:
# смотрим, сколько памяти использует столбец date
date = optimized_gl.date
print(mem_usage(date))
date.head()

0.6558952331542969 mb


0    18710504
1    18710505
2    18710506
3    18710508
4    18710509
Name: date, dtype: uint32

Мы преобразуем его с помощью функции **pandas.to_datetime()**, используя
параметр format, чтобы задать формат хранения наших дат YYYY-MM-DD.

In [19]:
# преобразуем столбец date в тип datetime
optimized_gl['date'] = pd.to_datetime(date, format='%Y%m%d')

# смотрим, сколько памяти использует столбец date
print(mem_usage(optimized_gl))
optimized_gl.date.head()

102.51544857025146 mb


0   1871-05-04
1   1871-05-05
2   1871-05-06
3   1871-05-08
4   1871-05-09
Name: date, dtype: datetime64[ns]

#### Задаем типы во время считывания данных
До сих пор мы рассматривали способы уменьшения объема памяти, работая
с **уже существующим датафреймом**. Сначала считывая датафрейм, а затем
перебирая разные способы уменьшения использования памяти, мы смогли выяснить,
какой объем памяти мы можем выиграть, применив ту или иную оптимизацию.
Однако, как мы уже говорили ранее, нам часто не хватает памяти
для представления всех значений в наборе данных. **Как мы можем применить
методы оптимизации памяти, если мы изначально даже не можем создать датафрейм?**

К счастью, мы можем указать оптимальные типы столбцов в ходе считывания набора
данных. Функция **read_csv()** имеет несколько параметров, которые позволяют
нам это делать. Параметр dtype принимает словарь, в котором ключами будут
строковые имена столбцов, а значениями – типы столбцов.

In [35]:
# создаем словарь, в котором ключами будут имена столбцов, а значениями – типы столбцов
dtypes = optimized_gl.drop('date', axis='columns').dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

# вместо того чтобы печатать 161 столбец, выберем 10 пар "ключ-значение" из словаря
# и красиво распечатаем их, используя prettyprint
preview = {key:value for key,value in list(column_types.items())[:10]}
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

{   'day_of_week': 'category',
    'h_game_number': 'uint8',
    'h_league': 'category',
    'h_name': 'category',
    'h_score': 'uint8',
    'number_of_game': 'uint8',
    'v_game_number': 'uint8',
    'v_league': 'category',
    'v_name': 'category',
    'v_score': 'uint8'}


In [37]:
# считываем данные с нужными нам типами
read_and_optimized = pd.read_csv('Notebooks/Data/game_logs.csv',
                                dtype=column_types,
                                parse_dates=['date'],
                                infer_datetime_format=True)

# смотрим использование памяти
print(mem_usage(read_and_optimized))
read_and_optimized.head()

102.5078296661377 mb


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,1871-05-04,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,1871-05-05,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,1871-05-06,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,1871-05-08,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,1871-05-09,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y
